In [1]:
import numpy as np
import json
import os

In [3]:
# read multiplication table file
# Windows infile
#infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '\z2_s4_001.json', 'r')]
# OS infile
infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '/z2_s4_001.json', 'r')]
mixer_index = infile[0]
M = infile[1] #multipliers
T = infile[2] #multiplication table

assert T.shape[0] == T.shape[1] == T.shape[2]
len_T = T.shape[0]
print("Number of conjugacy classes: {}" .format(len_T))


assert M.shape[1] == len_T
len_M = M.shape[0]
print("Number of multipliers: {}" .format(len_M))

#print("Multiplier Indices: {}" .format(M))

Number of conjugacy classes: 33
Number of multipliers: 9


In [4]:
print(M)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0 -1  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0 -1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0 -1  0  1]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1 -1  0  0  0
   0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   1  0  0  0 -2  0  0  0  1]
 [-1  0  0  1  2  0  0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0 -1  0
   0  0  0  0  0  0  0  0  1]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0  0 -1  0  0
   0  0  0  0  0  0  0  0  1]
 [-1  0  0  1  0  2  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0
   0 -1  0  0  0  0  0  0  1]
 [ 0 -1  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0 -1
   0  0  0 -2  0  0  0  0  1]]


In [5]:
# Convert multplication table from 3D array of integers to 3D list of bin
def array_to_list(X):
    return X.tolist()

M = M.astype(int).tolist()
T = T.astype(int)

In [6]:
nbytes = 2
PAD = '0'

def pad_string(m, len_T, PAD):
    pad_length = (len_T - len(m) % len_T)
    pad = PAD*pad_length
    return m + pad, pad_length

def string_to_byte_sequence(m, nbytes):
    #convert string to binary sequence
    #print(bin_version)
    return ''.join(format(ord(i), '08b') for i in m)

def byte_sequence_chunk(m, nbytes):
    #seperate byte sequence into list of 'chunks' of length 'bits_per_coefficient'
    bits_per_coefficient = nbytes * 2
    res = []
    for start in range(0, len(m), bits_per_coefficient):
        res.append(m[start:start + bits_per_coefficient])
    return res

def byte_sequence_to_binary_sequence(m):
    #interpret each byte chunk as binary integer 
    return [int(e,2) for e in m]

def binary_block_sequence(m, len_T):
    #seperate chunks into blocks of length 'len_T'
    return [m[i:i + len_T] for i in range(0, len(m), len_T)]

def binary_block_sequence_to_byte_sequence(m, nbytes):
    res = ''
    chunk_size = nbytes*2
    for i in range(len(m)):
        for j in range(len(m[i])):
            byte = str(bin(m[i][j])[2:].zfill(chunk_size))
            res += byte
    return res

def byte_to_pad_string(m):
    return ''.join(chr(int(m[i*8:i*8+8],2)) for i in range(len(m)//8))

def unpad_string(m, pad_length):
    return m[:len(m)-pad_length]
    

In [7]:
def string_to_bring(m, len_T, nbytes = 2, PAD = '0'):
    m1, pad_length = pad_string(m, len_T, PAD)
    m2 = string_to_byte_sequence(m1, nbytes)
    m3 = byte_sequence_chunk(m2, nbytes)
    m4 = byte_sequence_to_binary_sequence(m3)
    m5 = binary_block_sequence(m4, len_T)
    return m5, pad_length

def bring_to_string(m, len_T, pad_length, nbytes = 2, PAD = '0'):
    m1 = binary_block_sequence_to_byte_sequence(m, nbytes)
    m2 = byte_to_pad_string(m1)
    m3 = unpad_string(m2, pad_length)
    return m3

In [8]:
# Demonstration
m = 'abcdefg'
print("String: %s; Length: %s" % (m, len(m)))
m1, pad_length = pad_string(m, len_T, PAD)
print("Padded String: %s; Length: %s" % (m1, len(m1)))
print("Pad Length: %s" % (pad_length))
m2 = string_to_byte_sequence(m1, nbytes)
print("Byte Sequence: %s; Length: %s" % (m2, len(m2)))
m3 = byte_sequence_chunk(m2, nbytes)
print("Byte Sequence Chunks: %s" % (m3))
m4 = byte_sequence_to_binary_sequence(m3)
print("Binary Sequence: %s" % (m4))
m5 = binary_block_sequence(m4, len_T)
print("Binary Sequence Blocks: %s" % (m5))
m6 = binary_block_sequence_to_byte_sequence(m5, nbytes)
print("Byte Sequence: %s; Length: %s" % (m6, len(m6)))
m7 = byte_to_pad_string(m6)
print("Reconstructed Padded String: %s" % (m7))
m8 = unpad_string(m7, pad_length)
print("Reconstructed String: %s" % (m8))

String: abcdefg; Length: 7
Padded String: abcdefg00000000000000000000000000; Length: 33
Pad Length: 26
Byte Sequence: 011000010110001001100011011001000110010101100110011001110011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000; Length: 264
Byte Sequence Chunks: ['0110', '0001', '0110', '0010', '0110', '0011', '0110', '0100', '0110', '0101', '0110', '0110', '0110', '0111', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000']
Binary Sequence: [6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6,

In [10]:
def brmult_list(T, L, sel = "none"):
    #L: ring elements to be multiplied
    #T: multiplication table
    if sel == "none":
        pass
    else:
        L = L[sel,:]
        
    for l in L:
        if l == L[0]: # initialize product
            chi = l
        else:
            temp = np.zeros(len(l))
            for i in range(len(l)):
                for j in range(len(l)):
                    # chi[i]*l[j] is the appropriate coefficient
                    # T[i][j] is the appropriate ring element
                    temp+= chi[i]*l[j]*T[i][j]
            chi = temp
            #print("-------------------------------------------------------------")
            #print(chi) 
    return chi.astype(int).tolist()

In [18]:
msg = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatu'
bring_msg, pad_length = string_to_bring(msg, len_T)
print(bring_msg)

[[4, 12, 6, 15, 7, 2, 6, 5, 6, 13, 2, 0, 6, 9, 7, 0, 7, 3, 7, 5, 6, 13, 2, 0, 6, 4, 6, 15, 6, 12, 6, 15, 7], [2, 2, 0, 7, 3, 6, 9, 7, 4, 2, 0, 6, 1, 6, 13, 6, 5, 7, 4, 2, 12, 2, 0, 6, 3, 6, 15, 6, 14, 7, 3, 6, 5], [6, 3, 7, 4, 6, 5, 7, 4, 7, 5, 7, 2, 2, 0, 6, 1, 6, 4, 6, 9, 7, 0, 6, 9, 7, 3, 6, 3, 6, 9, 6, 14, 6], [7, 2, 0, 6, 5, 6, 12, 6, 9, 7, 4, 2, 12, 2, 0, 7, 3, 6, 5, 6, 4, 2, 0, 6, 4, 6, 15, 2, 0, 6, 5, 6, 9], [7, 5, 7, 3, 6, 13, 6, 15, 6, 4, 2, 0, 7, 4, 6, 5, 6, 13, 7, 0, 6, 15, 7, 2, 2, 0, 6, 9, 6, 14, 6, 3, 6], [9, 6, 4, 6, 9, 6, 4, 7, 5, 6, 14, 7, 4, 2, 0, 7, 5, 7, 4, 2, 0, 6, 12, 6, 1, 6, 2, 6, 15, 7, 2, 6, 5], [2, 0, 6, 5, 7, 4, 2, 0, 6, 4, 6, 15, 6, 12, 6, 15, 7, 2, 6, 5, 2, 0, 6, 13, 6, 1, 6, 7, 6, 14, 6, 1, 2], [0, 6, 1, 6, 12, 6, 9, 7, 1, 7, 5, 6, 1, 2, 14, 2, 0, 5, 5, 7, 4, 2, 0, 6, 5, 6, 14, 6, 9, 6, 13, 2, 0], [6, 1, 6, 4, 2, 0, 6, 13, 6, 9, 6, 14, 6, 9, 6, 13, 2, 0, 7, 6, 6, 5, 6, 14, 6, 9, 6, 1, 6, 13, 2, 12, 2], [0, 7, 1, 7, 5, 6, 9, 7, 3, 2, 0, 6, 14, 6, 15, 7, 3

In [19]:
multiplier = brmult_list(T,M)
print(multiplier)

[1, -1, 2, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -2, -2, -1, -1, -1, 1]


In [20]:
def bring_encryption(T,multiplier,m):
    #T: Multiplication table
    #multiplier: multiplier element in bring
    #m: bring representation of msg
    return [brmult_list(T,[e,multiplier]) for e in bring_msg]

In [21]:
encrypted_bring_msg = bring_encryption(T, multiplier, bring_msg)
print(bring_msg)
print(encrypted_bring_msg)

[[4, 12, 6, 15, 7, 2, 6, 5, 6, 13, 2, 0, 6, 9, 7, 0, 7, 3, 7, 5, 6, 13, 2, 0, 6, 4, 6, 15, 6, 12, 6, 15, 7], [2, 2, 0, 7, 3, 6, 9, 7, 4, 2, 0, 6, 1, 6, 13, 6, 5, 7, 4, 2, 12, 2, 0, 6, 3, 6, 15, 6, 14, 7, 3, 6, 5], [6, 3, 7, 4, 6, 5, 7, 4, 7, 5, 7, 2, 2, 0, 6, 1, 6, 4, 6, 9, 7, 0, 6, 9, 7, 3, 6, 3, 6, 9, 6, 14, 6], [7, 2, 0, 6, 5, 6, 12, 6, 9, 7, 4, 2, 12, 2, 0, 7, 3, 6, 5, 6, 4, 2, 0, 6, 4, 6, 15, 2, 0, 6, 5, 6, 9], [7, 5, 7, 3, 6, 13, 6, 15, 6, 4, 2, 0, 7, 4, 6, 5, 6, 13, 7, 0, 6, 15, 7, 2, 2, 0, 6, 9, 6, 14, 6, 3, 6], [9, 6, 4, 6, 9, 6, 4, 7, 5, 6, 14, 7, 4, 2, 0, 7, 5, 7, 4, 2, 0, 6, 12, 6, 1, 6, 2, 6, 15, 7, 2, 6, 5], [2, 0, 6, 5, 7, 4, 2, 0, 6, 4, 6, 15, 6, 12, 6, 15, 7, 2, 6, 5, 2, 0, 6, 13, 6, 1, 6, 7, 6, 14, 6, 1, 2], [0, 6, 1, 6, 12, 6, 9, 7, 1, 7, 5, 6, 1, 2, 14, 2, 0, 5, 5, 7, 4, 2, 0, 6, 5, 6, 14, 6, 9, 6, 13, 2, 0], [6, 1, 6, 4, 2, 0, 6, 13, 6, 9, 6, 14, 6, 9, 6, 13, 2, 0, 7, 6, 6, 5, 6, 14, 6, 9, 6, 1, 6, 13, 2, 12, 2], [0, 7, 1, 7, 5, 6, 9, 7, 3, 2, 0, 6, 14, 6, 15, 7, 3

In [22]:
decrypted_bring_msg = [brmult_list(T,[e,multiplier]) for e in encrypted_bring_msg]
print(decrypted_bring_msg)


[[4, 12, 6, 15, 7, 2, 6, 5, 6, 13, 2, 0, 6, 9, 7, 0, 7, 3, 7, 5, 6, 13, 2, 0, 6, 4, 6, 15, 6, 12, 6, 15, 7], [2, 2, 0, 7, 3, 6, 9, 7, 4, 2, 0, 6, 1, 6, 13, 6, 5, 7, 4, 2, 12, 2, 0, 6, 3, 6, 15, 6, 14, 7, 3, 6, 5], [6, 3, 7, 4, 6, 5, 7, 4, 7, 5, 7, 2, 2, 0, 6, 1, 6, 4, 6, 9, 7, 0, 6, 9, 7, 3, 6, 3, 6, 9, 6, 14, 6], [7, 2, 0, 6, 5, 6, 12, 6, 9, 7, 4, 2, 12, 2, 0, 7, 3, 6, 5, 6, 4, 2, 0, 6, 4, 6, 15, 2, 0, 6, 5, 6, 9], [7, 5, 7, 3, 6, 13, 6, 15, 6, 4, 2, 0, 7, 4, 6, 5, 6, 13, 7, 0, 6, 15, 7, 2, 2, 0, 6, 9, 6, 14, 6, 3, 6], [9, 6, 4, 6, 9, 6, 4, 7, 5, 6, 14, 7, 4, 2, 0, 7, 5, 7, 4, 2, 0, 6, 12, 6, 1, 6, 2, 6, 15, 7, 2, 6, 5], [2, 0, 6, 5, 7, 4, 2, 0, 6, 4, 6, 15, 6, 12, 6, 15, 7, 2, 6, 5, 2, 0, 6, 13, 6, 1, 6, 7, 6, 14, 6, 1, 2], [0, 6, 1, 6, 12, 6, 9, 7, 1, 7, 5, 6, 1, 2, 14, 2, 0, 5, 5, 7, 4, 2, 0, 6, 5, 6, 14, 6, 9, 6, 13, 2, 0], [6, 1, 6, 4, 2, 0, 6, 13, 6, 9, 6, 14, 6, 9, 6, 13, 2, 0, 7, 6, 6, 5, 6, 14, 6, 9, 6, 1, 6, 13, 2, 12, 2], [0, 7, 1, 7, 5, 6, 9, 7, 3, 2, 0, 6, 14, 6, 15, 7, 3

In [23]:
reconstructed_msg = bring_to_string(decrypted_bring_msg, len_T, pad_length)
print(reconstructed_msg)

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatu
